In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import matplotlib as mpl
from tqdm import tqdm


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

import hasasia.sim as hsim
import os


Set up

In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 10

BUILD_ARRAYS = False
SAVEFIG = True
TOL=0.01
MAXBADS=5

NVARS = 21
# NVARS = 6

NPSRS = 40
NSKIES = 100
# NSKIES = 15

SHOW_GW=True

Get param names

In [ ]:
pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=SHAPE, seed=None)
param_names = pspace.param_names

Get data and dets file functions

In [ ]:

filename = '/Users/emigardiner/GWs/holodeck/output/anatomy_7GW/mmb_mamp_log10_v21_r500_shapeNone/detstats_s100_r2w1.0_rg-1.5.npz'
file = np.load(filename, allow_pickle=True)
print(file.files)

In [ ]:
def get_data(
        target, nvars=NVARS, nreals=NREALS, nskies=NSKIES, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz'     
):
    load_data_from_file = path+f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}/data_params.npz' 
    load_dets_from_file = path+f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}/detstats_s{nskies}' 

    if red_gamma is not None and red2white is not None:
        load_dets_from_file = load_dets_from_file+f'_r2w{red2white:.1f}_rg{red_gamma:.1f}'
    else:
        load_dets_from_file = load_dets_from_file+f'_white'
    load_dets_from_file = load_dets_from_file+'.npz'

    if os.path.exists(load_data_from_file) is False:
        err = f"load data file '{load_data_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    if os.path.exists(load_dets_from_file) is False:
        err = f"load dets file '{load_dets_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    file = np.load(load_data_from_file, allow_pickle=True)
    data = file['data']
    params = file['params']
    file.close()
    print(target, "got data")
    file = np.load(load_dets_from_file, allow_pickle=True)
    print(target, "loaded dets")
    print(file.files)
    dsdat = file['dsdat']
    file.close()

    return data, params, dsdat

def get_ratio_arrays(
        target, nvars=NVARS, nreals=NREALS, nskies=NSKIES, shape=SHAPE, debug=False,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata',
    red=True,     
    ):
    filename = path+f'/ratio_arrays_{target}_v{nvars}_r{nreals}_s{nskies}_shape{str(shape)}.npz'
    file = np.load(filename)
    if debug: print(f"{filename}\n{file.files}")
    xx = file['xx_params']
    y0p0 = file['yy_ratio']
    if red:
        y1p5 = file['y1p5_ratio']
        y3p0 = file['y3p0_ratio']
    file.close()
    if red:
        return xx, y0p0, y1p5, y3p0, #y1p5, y3p0
    else:
        return xx, y0p0, 

In [ ]:
path = '/Users/emigardiner/GWs/holodeck/output/anatomy_7GW/figdata'
for target in ['gsmf_phi0', 'gsmf_mchar0_log10', 'mmb_mamp_log10', 'mmb_scatter_dex']:
    filename = path+f'/ratio_arrays_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz'
    file = np.load(filename)
    print(f"{filename}\n{file.files}")

In [ ]:
if True:
# if BUILD_ARRAYS:
    # targets = [
    #     'gsmf_phi0',  'mmb_mamp_log10', 'hard_time',  
    #     'gsmf_mchar0_log10', 'mmb_scatter_dex', 'hard_gamma_inner']
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz'  
    targets = [
        # 'gsmf_phi0',
        # 'gsmf_mchar0_log10', 
        # 'mmb_mamp_log10', 
        # 'mmb_scatter_dex',
        'hard_time'
    ]
    # path = '/Users/emigardiner/GWs/holodeck/output/anatomy_7GW'  
    figpath = '/Users/emigardiner/GWs/holodeck/output/anatomy_7GW/figdata'  
    for target in tqdm(targets):
        print(target)

        # white noise only
        data, params, dsdat = get_data(target, path=path)
        xx=[]
        yy=[]
        for pp, par in enumerate(params):
            xx.append(params[pp][target])
            dp_bg = np.repeat(dsdat[pp]['dp_bg'], NSKIES).reshape(NREALS, NSKIES)
            dp_ss = dsdat[pp]['ev_ss']
            yy.append(dp_ss/dp_bg)

        # red_gamma = -1.5
        data, params, dsdat = get_data(target, red_gamma=-1.5, red2white=1.0, path=path)
        y1p5=[]
        for pp, par in enumerate(params):
            dp_bg = np.repeat(dsdat[pp]['dp_bg'], NSKIES).reshape(NREALS, NSKIES)
            dp_ss = dsdat[pp]['ev_ss']
            y1p5.append(dp_ss/dp_bg)

        # red_gamma = -3.0
        data, params, dsdat = get_data(target, red_gamma=-3.0, red2white=1, path=path)
        y3p0=[]
        for pp, par in enumerate(params):
            dp_bg = np.repeat(dsdat[pp]['dp_bg'], NSKIES).reshape(NREALS, NSKIES)
            dp_ss = dsdat[pp]['ev_ss']
            y3p0.append(dp_ss/dp_bg)

            
        np.savez(figpath+f'/ratio_arrays_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz', 
                xx_params = xx, yy_ratio = yy, y1p5_ratio = y1p5, y3p0_ratio=y3p0)
                # xx_params = xx, yy_ratio = yy, y3p0_ratio=y3p0)

In [ ]:
filename = '/Users/emigardiner/GWs/holodeck/output/anatomy_7GW/mmb_mamp_log10_v21_r500_shapeNone/detstats_s100_r2w1.0_rg-3.0.npz'
file = np.load(filename)
print(file.files)

# Red Noise

In [ ]:
ylabel = plot.LABEL_DPRATIO
targets = [
    'gsmf_phi0',  'mmb_mamp_log10', 'hard_time',  
    'gsmf_mchar0_log10', 'mmb_scatter_dex', 'hard_gamma_inner']
col0p0 = [
    '#336948', '#9e5c41', '#2d839f',
    '#336948', '#9e5c41', '#2d839f',
]
col1p5 = [
    '#4da169', '#e67739', '#35b6e0',
    '#4da169', '#e67739', '#35b6e0',
]
col3p0 = [
    '#8fcf91', '#fda363', '#38cbff',
    '#8fcf91', '#fda363', '#38cbff',
]

col_gw = [
    'k', 'k', 'k',
    'k', 'k', 'k'
]

fig, axs = plot.figax_double(nrows=2, ncols=3, sharey=True, sharex=False, xscale='linear')
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii == 0 or ii == 3:
        ax.set_ylabel(ylabel)
    xx, yy0p0, yy1p5, yy3p0 = get_ratio_arrays(targets[ii], red=True)
    if SHOW_GW:
        if ii%3 != 2:
            xx_gw, yy_gw, y1p5_gw, y3p3_gw = get_ratio_arrays(targets[ii], red=True,
                            path='/Users/emigardiner/GWs/holodeck/output/anatomy_7GW/figdata')
        else:
            xx_gw = xx
            yy_gw = yy_gw[int(NVARS/2)]
            yy_gw = np.repeat(yy_gw, 21).reshape(NREALS*NSKIES, NVARS)
            yy_gw = np.swapaxes(yy_gw, 0, 1).reshape(NVARS, NREALS, NSKIES,)
    
    col=col0p0[ii]
    # for pp in [50, 95]:
    for pp in [68]: 
        med, *conf = np.percentile(yy0p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col0p0[ii], linestyle='-')
        ax.fill_between(xx, *conf, color=col0p0[ii], alpha=0.25)

        med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='-')
        # ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='--')

        med, *conf = np.percentile(yy1p5, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col1p5[ii], linestyle='--')
        ax.fill_between(xx, *conf, color=col1p5[ii], alpha=0.25)

        med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='--')
        # ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='--')

        med, *conf = np.percentile(yy3p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col3p0[ii], linestyle=':')
        ax.fill_between(xx, *conf, color=col3p0[ii], alpha=0.25)

        med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle=':')
        # ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='--')


handles = [mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=0.0$', color = col0p0[0]),
           mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=-1.5$', color = col1p5[0]),
           mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=-3.0$', color = col3p0[0])]
# axs[0,0].legend(handles=handles, loc='upper left', ncol=2)

plt.subplots_adjust(wspace=0)

# fig.tight_layout()

if SAVEFIG:
    savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
    if SHOW_GW:
        savename = savename+f'/ratio_vs_var{NVARS}_r2w_pp50.png'
    else:
        savename = savename+f'/ratio_vs_var{NVARS}_r2w_gw.png'
    # savename = savename+'/ratio_vs_var6_wn_pp5095.png'
    fig.savefig(savename, dpi=100)

# White noise with GW-Only

In [ ]:
ylabel = plot.LABEL_DPRATIO
targets = [
    'gsmf_phi0',  'mmb_mamp_log10', 'hard_time',  
    'gsmf_mchar0_log10', 'mmb_scatter_dex', 'hard_gamma_inner']
col0p0 = [
    '#336948', '#9e5c41', '#2d839f',
    '#336948', '#9e5c41', '#2d839f',
]
# col1p5 = [
#     '#4da169', '#e67739', '#35b6e0',
#     '#4da169', '#e67739', '#35b6e0',
# ]
# col3p0 = [
#     '#8fcf91', '#fda363', '#38cbff',
#     '#8fcf91', '#fda363', '#38cbff',
# ]
col_gw = [
    'k', 'k', 'k',
    'k', 'k', 'k'
]

fig, axs = plot.figax_double(nrows=2, ncols=3, sharey=True, sharex=False, xscale='linear')
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii == 0 or ii == 3:
        ax.set_ylabel(ylabel)
    xx, yy0p0, = get_ratio_arrays(targets[ii], red=False)

    col=col0p0[ii]
    for pp in [50, 95]:
    # for pp in [50]: 
        med, *conf = np.percentile(yy0p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col0p0[ii])
        ax.fill_between(xx, *conf, color=col0p0[ii], alpha=0.25)

    if ii%3 != 2:
        xx_gw, yy_gw = get_ratio_arrays(targets[ii], red=False,
                                        path='/Users/emigardiner/GWs/holodeck/output/anatomy_7GW/figdata')
        for pp in [50]:
            med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
            ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='--')
            ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='--')
        print(f"{ii=}, {yy_gw.shape}")
    else:
        xx_gw = xx
        yy_gw = yy_gw[int(NVARS/2)]
        yy_gw = np.repeat(yy_gw, 21).reshape(NREALS*NSKIES, NVARS)
        yy_gw = np.swapaxes(yy_gw, 0, 1).reshape(NVARS, NREALS, NSKIES,)
        print(f"{ii=}, {yy_gw.shape}")

        for pp in [50]:
            med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
            ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='--')
            ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='--')
        print(f"{ii=}, yy_gw.shape")

        

        # med, *conf = np.percentile(yy1p5, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        # ax.plot(xx, med, alpha=0.9, color=col1p5[ii])
        # ax.fill_between(xx, *conf, color=col1p5[ii], alpha=0.25)

        # med, *conf = np.percentile(yy3p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        # ax.plot(xx, med, alpha=0.9, color=col3p0[ii])
        # ax.fill_between(xx, *conf, color=col3p0[ii], alpha=0.25)

# handles = [mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=0.0$', color = col0p0[0]),
#            mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=-1.5$', color = col1p5[0]),
#            mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=-3.0$', color = col3p0[0])]
# axs[0,0].legend(handles=handles, loc='upper left', ncol=2)
plt.subplots_adjust(wspace=0)
# fig.tight_layout()

if SAVEFIG:
    savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
    # savename = savename+'/ratio_vs_var6_r2w_pp50.png'
    savename = savename+f'/ratio_vs_var{NVARS}_wn_gw.png'
    fig.savefig(savename, dpi=100)

In [ ]:
ylabel = plot.LABEL_EVSS
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
col0p0 = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]
# col1p5 = [
#     '#4da169', '#e67739', '#35b6e0',
#     '#4da169', '#e67739', '#35b6e0',
# ]
# col3p0 = [
#     '#8fcf91', '#fda363', '#38cbff',
#     '#8fcf91', '#fda363', '#38cbff',
# ]
col_gw = [
    'k', 'k', 'k',
    'k', 'k', 'k'
]

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, xscale='linear',
                             height=7)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii == 2:
        ax.set_ylabel(ylabel)

    ax.yaxis.set_major_locator(mpl.ticker.LogLocator(numticks=999))
    ax.yaxis.set_minor_locator(mpl.ticker.LogLocator(numticks=999, subs=(.2, .4, .6, .8)))

    xx, yy0p0, = get_ratio_arrays(targets[ii], red=False)
    yy0p0 *= 0.5

    col=col0p0[ii]
    for pp in [50, 95]:
    # for pp in [50]: 
        med, *conf = np.percentile(yy0p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col0p0[ii])
        ax.fill_between(xx, *conf, color=col0p0[ii], alpha=0.25)

    if ii<4:
        xx_gw, yy_gw = get_ratio_arrays(targets[ii], red=False,
                                        path='/Users/emigardiner/GWs/holodeck/output/anatomy_7GW/figdata')
        yy_gw *= 0.5
        for pp in [50]:
            med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
            ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='--')
            ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='--')
        print(f"{ii=}, {yy_gw.shape}")
    else:
        xx_gw = xx
        yy_gw = yy_gw[int(NVARS/2)]
        yy_gw = np.repeat(yy_gw, 21).reshape(NREALS*NSKIES, NVARS)
        yy_gw = np.swapaxes(yy_gw, 0, 1).reshape(NVARS, NREALS, NSKIES,)
        print(f"{ii=}, {yy_gw.shape}")

        for pp in [50]:
            med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
            ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='--')
            ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='--')
        print(f"{ii=}, yy_gw.shape")

        

        # med, *conf = np.percentile(yy1p5, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        # ax.plot(xx, med, alpha=0.9, color=col1p5[ii])
        # ax.fill_between(xx, *conf, color=col1p5[ii], alpha=0.25)

        # med, *conf = np.percentile(yy3p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        # ax.plot(xx, med, alpha=0.9, color=col3p0[ii])
        # ax.fill_between(xx, *conf, color=col3p0[ii], alpha=0.25)

# handles = [mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=0.0$', color = col0p0[0]),
#            mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=-1.5$', color = col1p5[0]),
#            mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=-3.0$', color = col3p0[0])]
# axs[0,0].legend(handles=handles, loc='upper left', ncol=2)
plt.subplots_adjust(wspace=0.05, hspace=0.35)
# fig.tight_layout()

if SAVEFIG:
    savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
    # savename = savename+'/ratio_vs_var6_r2w_pp50.png'
    savename = savename+f'/evss_vs_var{NVARS}_wn_gw_single.png'
    fig.savefig(savename, dpi=100)

In [ ]:
# ylabel = '$\langle N \\rangle _\mathrm{SS} / \gamma_\mathrm{BG}$'
# targets = [
#     'gsmf_phi0', 'gsmf_mchar0_log10', 
#       'mmb_mamp_log10',  'mmb_scatter_dex',
#       'hard_time',  'hard_gamma_inner']
# col0p0 = [
#     '#336948', '#336948', 
#     '#9e5c41', '#9e5c41', 
#     '#2d839f', '#2d839f',
# ]
# col1p5 = [
#     '#4da169', '#4da169',
#     '#e67739', '#e67739',
#     '#35b6e0', '#35b6e0',
     
# ]
# col3p0 = [
#     '#8fcf91', '#8fcf91',
#     '#fda363', '#fda363',
#     '#38cbff', '#38cbff',
# ]
# col_gw = [
#     'k', 'k', 'k',
#     'k', 'k', 'k'
# ]

# SHOW_GW = True
# SHOW_REDLINES = True

# fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, xscale='linear',
#                              height=7)
# for ii, ax in enumerate(axs.flatten()):
#     ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
#     if ii ==2:
#         ax.set_ylabel(ylabel)
#     xx, yy0p0, yy1p5, yy3p0 = get_ratio_arrays(targets[ii], red=True)
#     yy0p0 *= 0.5
#     yy1p5 *= 0.5
#     yy3p0 *= 0.5
#     col=col0p0[ii]
#     for pp in [50, 95]:
#     # for pp in [50]: 
#         med, *conf = np.percentile(yy0p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
#         ax.plot(xx, med, alpha=0.9, color=col0p0[ii])
#         ax.fill_between(xx, *conf, color=col0p0[ii], alpha=0.25)

#     if ii<4 and SHOW_GW:
#         xx_gw, yy_gw = get_ratio_arrays(targets[ii], red=False,
#                                         path='/Users/emigardiner/GWs/holodeck/output/anatomy_7GW/figdata')
#         yy_gw *= 0.5
#         for pp in [50]:
#             med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
#             ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='-')
#             ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='-')
#     elif SHOW_GW:
#         xx_gw = xx
#         yy_gw = yy_gw[int(NVARS/2)]
#         yy_gw = np.repeat(yy_gw, 21).reshape(NREALS*NSKIES, NVARS)
#         yy_gw = np.swapaxes(yy_gw, 0, 1).reshape(NVARS, NREALS, NSKIES,)
#         # print(f"{ii=}, {yy_gw.shape}")

#         for pp in [50]:
#             med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
#             ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='-')
#             ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='-')
#         # print(f"{ii=}, yy_gw.shape")

#     if SHOW_REDLINES:
#         for pp in [50,]:
#             # Red Noise Models
#             med, *conf = np.percentile(yy1p5, [50, 50-pp/2, 50+pp/2], axis=(1,2))
#             ax.plot(xx, med, alpha=0.9, color=col1p5[ii], linestyle='--')
#             # ax.fill_between(xx, *conf, color=col1p5[ii], alpha=0.25)
        
#             med, *conf = np.percentile(yy3p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
#             ax.plot(xx, med, alpha=0.9, color=col3p0[ii], linestyle=':')
#             # ax.fill_between(xx, *conf, color=col3p0[ii], alpha=0.25)


#             if SHOW_GW:
#                 med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
#                 # ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='--')
#                 # ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='--')

#                 med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
#                 # ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle=':')
        

# # handles = [mpl.lines.Line2D([0], [0], label='0.0', color = col0p0[0], linestyle='-'),
# #            mpl.lines.Line2D([0], [0], label='-1.5', color = col1p5[0], linestyle='--'),
# #            mpl.lines.Line2D([0], [0], label='-3.0', color = col3p0[0], linestyle=':')]
# # leg = fig.legend(handles=handles, bbox_to_anchor=(0.5,0.05), loc = 'center',
# #                 title='$\gamma_\mathrm{RN}$', ncols=4, 
# #                 fontsize=10, title_fontsize=12, handletextpad=0.1, labelspacing=0.2)
# # leg.get_title().set_position((-50,-10))
# plt.subplots_adjust(wspace=0.05, hspace=0.35)
# # fig.tight_layout()

# if SAVEFIG:
#     savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
#     # savename = savename+'/ratio_vs_var6_r2w_pp50.png'
#     savename = savename+f'/evss_var{NVARS}_gw_single_rnlines.png'
#     fig.savefig(savename, dpi=100, bbox_inches='tight')

# Red Noise, GW, Single

In [ ]:
def repeat_mid_yy(yy):
    yy = yy[int(NVARS/2)]
    yy = np.repeat(yy, NVARS).reshape(NREALS*NSKIES, NVARS)
    yy = np.swapaxes(yy, 0, 1).reshape(NVARS, NREALS, NSKIES,)
    return yy

In [ ]:
ylabel = plot.LABEL_EVSS
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
col0p0 = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]
col1p5 = [
    '#4da169', '#4da169',
    '#e67739', '#e67739',
    '#35b6e0', '#35b6e0',
     
]
col3p0 = [
    '#8fcf91', '#8fcf91',
    '#fda363', '#fda363',
    '#38cbff', '#38cbff',
]
col_gw = [
    'k', 'k', 'k',
    'k', 'k', 'k'
]

SHOW_GW = True

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, xscale='linear',
                             height=7)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii ==2:
        ax.set_ylabel(ylabel)
    ax.tick_params(axis='y', which='both', right=True, left=True, direction='in')
    ax.tick_params(axis='x', which='both', top=True, direction='in')
    
    ax.yaxis.set_major_locator(mpl.ticker.LogLocator(numticks=999))
    ax.yaxis.set_minor_locator(mpl.ticker.LogLocator(numticks=999, subs=(.2, .4, .6, .8)))



    xx, yy0p0, yy1p5, yy3p0 = get_ratio_arrays(targets[ii], red=True)
    yy0p0 *= 0.5
    yy1p5 *= 0.5
    yy3p0 *= 0.5
    col=col0p0[ii]
    for pp in [68,]:
    # for pp in [50]: 
        med, *conf = np.percentile(yy0p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col0p0[ii])
        ax.fill_between(xx, *conf, color=col0p0[ii], alpha=0.25)

    if SHOW_GW:
        if ii<4:
            xx_gw, gw_0p0, gw_1p5, gw_3p0 = get_ratio_arrays(targets[ii], red=True,
                                            path='/Users/emigardiner/GWs/holodeck/output/anatomy_7GW/figdata')
            gw_0p0 *= 0.5
            gw_1p5 *= 0.5
            gw_3p0 *= 0.5
            for pp in [68,]:
                med, *conf = np.percentile(gw_0p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
                ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='-')
                # ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='-')
        else:
            xx_gw = xx
            gw_0p0 = repeat_mid_yy(gw_0p0)
            gw_1p5 = repeat_mid_yy(gw_1p5)
            gw_3p0 = repeat_mid_yy(gw_3p0)


        for pp in [68,]:
            med, *conf = np.percentile(gw_0p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
            ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='-')
            # ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='-')

            med, *conf = np.percentile(gw_1p5, [50, 50-pp/2, 50+pp/2], axis=(1,2))
            ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='--')
            # ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='--')

            med, *conf = np.percentile(gw_3p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
            ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle=':')
            # ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle=':')


    for pp in [50,]:
        # Red Noise Models
        med, *conf = np.percentile(yy1p5, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col1p5[ii], linestyle='--')
        ax.fill_between(xx, *conf, color=col1p5[ii], alpha=0.25)
       
        med, *conf = np.percentile(yy3p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col3p0[ii], linestyle=':')
        ax.fill_between(xx, *conf, color=col3p0[ii], alpha=0.25)

        

# handles = [mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=0.0$', color = col0p0[0]),
#            mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=-1.5$', color = col1p5[0]),
#            mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=-3.0$', color = col3p0[0])]
# axs[0,0].legend(handles=handles, loc='upper left', ncol=2)
plt.subplots_adjust(wspace=0.05, hspace=0.35)
# fig.tight_layout()

if SAVEFIG:
    savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
    # savename = savename+'/ratio_vs_var6_r2w_pp50.png'
    savename = savename+f'/evss_vs_var{NVARS}_rn_gw_single_68ci.png'
    fig.savefig(savename, dpi=100)